In [1]:
import os, pathlib

file_path = os.path.dirname(os.path.realpath("__file__"))
home_dir = pathlib.Path(file_path).parent.parent
os.chdir(home_dir)

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from src.utils import SAVED_PICS_PATH, keras_dataset_image_preprocessing, DENSE_LAYER_NAME, CONV_LAYER_NAME, ALPHAS
from src.utils import SAVED_CLASSIFIER_PATH_MNIST, SAVED_CLADEC_PATH_MNIST, SAVED_REFAE_PATH_MNIST
from src.cladec import ClaDec, RefAE

(train_images, train_labels_c), (test_images, test_labels_c) = keras_dataset_image_preprocessing(keras.datasets.mnist)
SAVED_EVAL_PICS_PATH = SAVED_PICS_PATH.joinpath('fashion_mnist')

tf.random.set_seed(1234)
np.random.seed(1234)

In [3]:
classifier = keras.models.load_model(SAVED_CLASSIFIER_PATH_MNIST)
classifier.evaluate(test_images, test_labels_c)

2022-07-10 12:01:50.802111: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 12:01:50.838624: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 12:01:50.838984: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 12:01:50.839723: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

313/313 [==============================] - 2s 4ms/step - loss: 0.0414 - categorical_accuracy: 0.9905


[0.04137476161122322, 0.9904999732971191]

In [4]:
decoder_dense = ClaDec.create_128_dense_decoder()
for alpha in ALPHAS:
    cladec_path = SAVED_CLADEC_PATH_MNIST.joinpath(f'{alpha:2}')
    cladec_dense = ClaDec.create_from_weights(cladec_path.joinpath(DENSE_LAYER_NAME).joinpath(DENSE_LAYER_NAME),
                                              classifier, DENSE_LAYER_NAME, alpha / 100, decoder_dense)
    print("Evaluating ", cladec_path.joinpath(DENSE_LAYER_NAME))
    cladec_dense.evaluate(test_images, test_labels_c)

Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 0/my_dense
313/313 [==============================] - 2s 6ms/step - loss: 0.0266 - reconstruction_loss: 0.0264 - classification_loss: 0.1374
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 1/my_dense
313/313 [==============================] - 2s 6ms/step - loss: 0.0273 - reconstruction_loss: 0.0265 - classification_loss: 0.0729
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 5/my_dense
313/313 [==============================] - 2s 6ms/step - loss: 0.0309 - reconstruction_loss: 0.0287 - classification_loss: 0.0739
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/10/my_dense
313/313 [==============================] - 2s 6ms/step - loss: 0.0341 - reconstruction_loss: 0.0307 - classification_loss: 0.0649
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/25/my_dense
313/313 [==============================] - 2s 6ms/step - loss: 0.0522 - rec

In [5]:
decoder_conv = ClaDec.create_64_conv_decoder()
for alpha in ALPHAS:
    cladec_path = SAVED_CLADEC_PATH_MNIST.joinpath(f'{alpha:2}')
    cladec_conv = ClaDec.create_from_weights(cladec_path.joinpath(CONV_LAYER_NAME).joinpath(CONV_LAYER_NAME),
                                             classifier, CONV_LAYER_NAME, alpha / 100, decoder_conv)
    print("Evaluating ", cladec_path.joinpath(CONV_LAYER_NAME))
    cladec_conv.evaluate(test_images, test_labels_c)

Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 0/my_conv
313/313 [==============================] - 2s 6ms/step - loss: 0.0018 - reconstruction_loss: 0.0018 - classification_loss: 0.0542
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 1/my_conv
313/313 [==============================] - 2s 6ms/step - loss: 0.0024 - reconstruction_loss: 0.0019 - classification_loss: 0.0475
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 5/my_conv
313/313 [==============================] - 3s 7ms/step - loss: 0.0043 - reconstruction_loss: 0.0022 - classification_loss: 0.0434
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/10/my_conv
313/313 [==============================] - 2s 6ms/step - loss: 0.0066 - reconstruction_loss: 0.0026 - classification_loss: 0.0426
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/25/my_conv
313/313 [==============================] - 2s 6ms/step - loss: 0.0174 - reconstr

In [6]:
# load RefAE model
refae_dense = RefAE.create_from_weights(SAVED_REFAE_PATH_MNIST.joinpath(DENSE_LAYER_NAME).joinpath(DENSE_LAYER_NAME),
                                        cladec_dense)
refae_conv = RefAE.create_from_weights(SAVED_REFAE_PATH_MNIST.joinpath(CONV_LAYER_NAME).joinpath(CONV_LAYER_NAME),
                                       cladec_conv)
# get reference reconstructions
print("Evaluating ", SAVED_REFAE_PATH_MNIST.joinpath(DENSE_LAYER_NAME))
refae_dense.evaluate(test_images)
print("Evaluating ", SAVED_REFAE_PATH_MNIST.joinpath(CONV_LAYER_NAME))
refae_conv.evaluate(test_images)

Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/refae/my_dense
313/313 [==============================] - 2s 4ms/step - reconstruction_loss: 0.0031
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/refae/my_conv
313/313 [==============================] - 1s 4ms/step - reconstruction_loss: 5.2530e-04


0.000525295443367213

# Train classifier on reconstructions

In [17]:
from src.create_models import train_classifier

train_reconstructions_refae_dense = refae_dense.predict(train_images)
train_reconstructions_cladec_dense = cladec_dense.predict(train_images)

test_reconstructions_refae_dense = refae_dense.predict(test_images)
test_reconstructions_cladec_dense = cladec_dense.predict(test_images)

313/313 [==============================] - 1s 3ms/step


In [11]:
classifier_refae_dense = train_classifier(train_reconstructions_refae_dense, train_labels_c, 1)

1875/1875 [==============================] - 12s 6ms/step - loss: 0.1715 - categorical_accuracy: 0.9485


In [13]:
classifier_refae_dense.evaluate(test_images, test_labels_c)

313/313 [==============================] - 1s 3ms/step - loss: 0.0571 - categorical_accuracy: 0.9817


[0.05708133429288864, 0.9817000031471252]

In [18]:
classifier_refae_dense.evaluate(test_reconstructions_refae_dense, test_labels_c)

313/313 [==============================] - 1s 4ms/step - loss: 0.0676 - categorical_accuracy: 0.9789


[0.06760738044977188, 0.9789000153541565]

In [14]:
classifier_cladec_99_dense = train_classifier(train_reconstructions_cladec_dense, train_labels_c, 1)

1875/1875 [==============================] - 12s 6ms/step - loss: 0.0693 - categorical_accuracy: 0.9828


In [15]:
classifier_cladec_99_dense.evaluate(test_images, test_labels_c)

313/313 [==============================] - 1s 4ms/step - loss: 9.6945 - categorical_accuracy: 0.1816


[9.694467544555664, 0.18160000443458557]

In [19]:
classifier_cladec_99_dense.evaluate(test_reconstructions_cladec_dense, test_labels_c)

313/313 [==============================] - 1s 4ms/step - loss: 0.0640 - categorical_accuracy: 0.9852


[0.063988097012043, 0.9851999878883362]